# Data Cleaning

In [1]:
import requests
import json
import pandas as pd
import datetime
import numpy as np

## Loading the data

In [2]:
filepath = "../data/raw/"

#data tables
creator_df = pd.read_csv(filepath + 'creator_df.csv')
nft_sales_df = pd.read_csv(filepath + 'nft_sales_df.csv')
nft_bid_df = pd.read_csv(filepath + 'nft_bid_df.csv')
nft_metadata_df = pd.read_csv(filepath + 'nft_metadata_df.csv')

# connectors
creator_nft_df = pd.read_csv(filepath + 'creator_nft_df.csv')
nft_nft_history_df = pd.read_csv(filepath + 'nft_nft_history_df.csv')

## Cleaning of creator_df data

In [3]:
creator_df.head()

,id,netRevenueInETH,netRevenuePendingInETH,netSalesInETH,netSalesPendingInETH
0,0xf74d1224931afa9cf12d06092c1eb1818d1e255c,1890.400000,0.0,2224.000000,0.0
1,0x4666fd1f89576e2d6fb3f2ecec5eefd3e1ba6b59,1329.278945,0.0,1624.651700,0.0
2,0x484ec62385e780f2460feac34864a77ba5a18134,513.400000,0.0,604.000000,0.0
3,0x0624d062ae9dd596de0384d37522cde46cd500d6,425.000000,0.0,500.000000,0.0
4,0x0f441cfad93287109f5ef834bf52f4aaaa8d8ffa,380.005263,0.0,447.065016,0.0


According the Foundation API, the 'netRevenuePendingInETH' and 'netSalesPendingInETH' refers to part of total ETH sales for minted NFTs which transactions have not been finalized.  

These columns are more indicative each creator/collectors choice in finalizing ETH transactions, and thus are not included in the analysis of ETH price of NFTs. 

In [4]:
creator_df.drop(['netRevenuePendingInETH', 'netSalesPendingInETH'], axis = 1, inplace = True);

'netRevenueInETH' refers to the value in ETH earned by each creator after subtracting intermediary fees such as ETH gas fees.  
'netSalesInETH' refers to the value in ETH for sales of NFTs by each creator.  

To check, a new column 'c_rs_ratio' is created to check that net sales is always larger than net revenue. 

In [5]:
creator_df['c_rs_ratio'] = creator_df['netRevenueInETH'] / creator_df['netSalesInETH']
creator_df[creator_df['c_rs_ratio'] > 1].shape

(0, 4)

Finally, the prefix "c" is added to each column name to signify info related to creator. The net sales and revenue columns can be used as a ranking system to rank creator by earnings, which could be a variable that affects NFT price.  

Both the sales and revenue columns are kept in as there might be additional factors such as gas price aside from the 15% service fee charged by Foundation.

In [6]:
new_column_names = ['c_id', 'c_revenue', 'c_sales', 'c_rs_ratio']
old_column_names = creator_df.columns
column_change = {old_column_names[col]:new_column_names[col] for col in range(len(old_column_names))}
creator_df.rename(columns = column_change, inplace = True);

In [7]:
creator_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6000 entries, 0 to 5999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   c_id        6000 non-null   object 
 1   c_revenue   6000 non-null   float64
 2   c_sales     6000 non-null   float64
 3   c_rs_ratio  6000 non-null   float64
dtypes: float64(3), object(1)
memory usage: 187.6+ KB


In [8]:
creator_df.describe().T

,count,mean,std,min,25%,50%,75%,max
c_revenue,6000.0,5.772355,34.988471,0.10,0.96815,1.75746,4.134612,1890.40
c_sales,6000.0,7.001000,41.823261,0.73,1.15000,2.11000,5.000000,2224.00
c_rs_ratio,6000.0,0.836972,0.059240,0.10,0.85000,0.85000,0.850000,0.85


In [9]:
creator_df.isnull().sum()

c_id          0
c_revenue     0
c_sales       0
c_rs_ratio    0
dtype: int64

## Cleaning of nft_sales_df data

In [10]:
nft_sales_df.head()

,dateMinted,id,isFirstSale,lastSalePriceInETH,netRevenueInETH,netRevenuePendingInETH,netSalesInETH,netSalesPendingInETH,percentSplit,tokenIPFSPath
0,1618452786,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,False,2224.0000,1890.40000,12677.174465,2224.0000,14914.3229,NaN,QmNeqUFf3WKcXFEHgxFCrf3t8jrxyCkev6LBtRz78Ygq9u...
1,1634633808,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100052,False,49.5002,42.07517,158.788500,49.5002,186.8100,NaN,QmaW2ukMCqyMNg5qqoYsusgbNL9KRnFwRKCkBt6u3AiyTN...
2,1634808905,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100995,False,33.0000,3.30000,0.000000,33.0000,0.0000,NaN,QmdCKx9Q5b1G4vjbJ2wKSkCo3sWoUrg2hm5QTifNag5nXj...
3,1634809019,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100997,True,NaN,0.00000,0.000000,0.0000,0.0000,NaN,QmbeU7NQaLKou2MnVDguPBJTuX1r6qAdmZ3CaSKGtE75iW...
4,1634809215,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100998,True,NaN,0.00000,0.000000,0.0000,0.0000,NaN,QmUZ7gQDvFUWXmqTBY7FHgqVBCvzvxfNSf6vEZdVffsGpM...


#### Convert epoch time to date time for date_minted

In [11]:
nft_sales_df['n_date_minted'] = [datetime.datetime.fromtimestamp(nft_sales_df['dateMinted'][x]) for x in range(len(nft_sales_df['dateMinted']))]

The "percentSplit" column contains more than 96% null and is removed. Pending ETH values are also removed.

In [12]:
nft_sales_df['percentSplit'].notnull().unique()

array([False,  True])

In [13]:
nft_sales_df.drop(['netRevenuePendingInETH', 'netSalesPendingInETH', 'percentSplit', 'dateMinted'], axis = 1, inplace = True)

In [14]:
new_column_names = ['n_id', 'n_isFirstSale', 'n_lastSalePriceInETH', 'n_revenue', 'n_sales', 'n_tokenIPFSPath', 'n_date_minted']
old_column_names = nft_sales_df.columns
column_change = {old_column_names[col]:new_column_names[col] for col in range(len(old_column_names))}
nft_sales_df.rename(columns = column_change, inplace = True);

### Checking to make sure there are no erroneous data for sales and revenue

In [15]:
nft_sales_df['n_rs_ratio'] = nft_sales_df['n_revenue'] / nft_sales_df['n_sales'] 
nft_sales_df[nft_sales_df['n_rs_ratio'] > 1].shape

(0, 8)

In [16]:
nft_sales_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58830 entries, 0 to 58829
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   n_id                  58830 non-null  object        
 1   n_isFirstSale         58830 non-null  bool          
 2   n_lastSalePriceInETH  34493 non-null  float64       
 3   n_revenue             58830 non-null  float64       
 4   n_sales               58830 non-null  float64       
 5   n_tokenIPFSPath       58830 non-null  object        
 6   n_date_minted         58830 non-null  datetime64[ns]
 7   n_rs_ratio            34486 non-null  float64       
dtypes: bool(1), datetime64[ns](1), float64(4), object(2)
memory usage: 3.2+ MB


In [17]:
nft_sales_df.describe().T

,count,mean,std,min,25%,50%,75%,max
n_lastSalePriceInETH,34493.0,1.191902,13.665802,0.0,0.2201,0.4500,1.000,2224.000000
n_revenue,58830.0,0.584517,8.900962,0.0,0.0000,0.1275,0.425,1890.400000
n_sales,58830.0,0.709031,10.488230,0.0,0.0000,0.1500,0.500,2224.000000
n_rs_ratio,34486.0,0.838544,0.078288,0.1,0.8500,0.8500,0.850,0.927778


In [18]:
nft_sales_df['n_lastSalePriceInETH'][nft_sales_df['n_lastSalePriceInETH'] == 0].count()

9

In [19]:
nft_sales_df[nft_sales_df['n_lastSalePriceInETH'] == 0]['n_tokenIPFSPath'][2658]

'QmcFQPZ25WwYein28QhU37qYUuTZmBKCwp5zhXkk4HWK3y/metadata.json'

It might be of interest to further gather the reserve price to start bidding

## Cleaning of nft metadata

In [20]:
nft_metadata_df.head()

,tokenIPFS,name,description,image
0,QmNeqUFf3WKcXFEHgxFCrf3t8jrxyCkev6LBtRz78Ygq9u...,"Stay Free (Edward Snowden, 2021)","This unique, signed work combines the entirety...",ipfs://QmTKdyzYbT9FzAs8mB4HGiLDR7o8maRP34J5iND...
1,QmaW2ukMCqyMNg5qqoYsusgbNL9KRnFwRKCkBt6u3AiyTN...,PORTAL 07,Possession of this PORTAL entitles you to memb...,ipfs://QmYChA4jtLhiMLPhDZv1t3xGZ1uj46yG4dtzCyg...
2,QmdCKx9Q5b1G4vjbJ2wKSkCo3sWoUrg2hm5QTifNag5nXj...,PORTAL 08,Possession of this PORTAL entitles you to memb...,ipfs://Qmbd3CzQMBPYeYBiMWe7U2bTWCyRusEjcMB3Ck8...
3,QmbeU7NQaLKou2MnVDguPBJTuX1r6qAdmZ3CaSKGtE75iW...,PORTAL 09,Possession of this PORTAL entitles you to memb...,ipfs://QmPMqidEE76so1tksfE1DMMheohB3NUzVVhZBx5...
4,QmUZ7gQDvFUWXmqTBY7FHgqVBCvzvxfNSf6vEZdVffsGpM...,PORTAL 10,Possession of this PORTAL entitles you to memb...,ipfs://QmZATpHmc3f5vxaVAzKCveEWdNR5jDUWY9Cyqv3...


In [21]:
new_column_names = ['m_tokenIPFS', 'm_name', 'm_desc', 'm_img']
old_column_names = nft_metadata_df.columns
column_change = {old_column_names[col]:new_column_names[col] for col in range(len(old_column_names))}
nft_metadata_df.rename(columns = column_change, inplace = True)

In [22]:
nft_metadata_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58720 entries, 0 to 58719
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   m_tokenIPFS  58720 non-null  object
 1   m_name       58717 non-null  object
 2   m_desc       57482 non-null  object
 3   m_img        58718 non-null  object
dtypes: object(4)
memory usage: 1.8+ MB


In [23]:
nft_metadata_df.describe().T

,count,unique,top,freq
m_tokenIPFS,58720,58720,QmNeqUFf3WKcXFEHgxFCrf3t8jrxyCkev6LBtRz78Ygq9u...,1
m_name,58717,53545,Serenity,28
m_desc,57482,53436,Ongoing series titled God's People. (2013-\n\n...,69
m_img,58718,57789,ipfs://undefined,13


In [24]:
nft_metadata_df.isnull().sum()

m_tokenIPFS       0
m_name            3
m_desc         1238
m_img             2
dtype: int64

## Cleaning of nft bid_data

In [25]:
nft_bid_df.head()

,amountInETH,date,event,id
0,444.0,1618595528,Bid,0x03b1c1227402f5b139f6c3d3c34ec1ffda707553a5fd...
1,2.0,1618512578,Listed,0x0425972b4e71e3e307f0b81b9a4b4d23f559b73d46e5...
2,122.0,1618527555,Bid,0x0708e2f65b149a1b6b429dbe366bd590786e86724c81...
3,611.0,1618598693,Bid,0x09ed6d047d08c3c531c5ca5842028db4a1a63017f057...
4,1337.0,1618600772,Bid,0x0d9fd52fa83ff321549aeb52fa488f69ccd95e7903a5...


#### Using Connector 2 to add nft_id for bid details (Left join with verification of one_to_one)

In [26]:
nft_nft_history_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327651 entries, 0 to 327650
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   nft_id          327651 non-null  object
 1   nft_history_id  327651 non-null  object
dtypes: object(2)
memory usage: 5.0+ MB


In [27]:
nft_bid_df = pd.merge(nft_bid_df, nft_nft_history_df, how = "left", left_on = "id", right_on = "nft_history_id", validate = "one_to_one")

#### Generate descriptive metrics for events (pivot then flatten back to dataframe)

In [28]:
nft_bid_df_pivot = pd.pivot_table(nft_bid_df,
                                  index = ["nft_id", "event"],
                                   values = ["amountInETH"],
                                   aggfunc = {"amountInETH":[min, max, np.mean, len]}
                                  )
nft_bid_df = pd.DataFrame(nft_bid_df_pivot.to_records())

#### Only the listed, bid, sold, and private sale events are preserved, where private sale is treated as equal to sale

In [29]:
nft_bid_df['event'].value_counts()

Minted                           58862
Listed                           55755
Bid                              33969
Settled                          33837
Sold                             33837
PriceChanged                     17356
Unlisted                          8700
Burned                            7036
Transferred                       6605
PrivateSale                        701
CreatorMigrated                    346
SellerMigrated                      85
OwnerMigrated                       44
CreatorPaymentAddressMigrated       16
Name: event, dtype: int64

In [30]:
nft_bid_df = nft_bid_df[nft_bid_df['event'].isin(['Bid', 'Listed', 'Sold','PrivateSale'])]

In [33]:
nft_bid_df['event'].replace('PrivateSale','Sold', inplace = True);

#### Pivot once more to reduce bid, list, and sale details down to one row per nft

In [35]:
nft_bid_df = pd.pivot_table(nft_bid_df,
                            index = "nft_id",
                            columns = "event")
nft_bid_df = pd.DataFrame(nft_bid_df.to_records())

#### Tidy up columns

In [36]:
list(nft_bid_df.columns)

['nft_id',
 '("(\'amountInETH\', \'len\')", \'Bid\')',
 '("(\'amountInETH\', \'len\')", \'Listed\')',
 '("(\'amountInETH\', \'len\')", \'Sold\')',
 '("(\'amountInETH\', \'max\')", \'Bid\')',
 '("(\'amountInETH\', \'max\')", \'Listed\')',
 '("(\'amountInETH\', \'max\')", \'Sold\')',
 '("(\'amountInETH\', \'mean\')", \'Bid\')',
 '("(\'amountInETH\', \'mean\')", \'Listed\')',
 '("(\'amountInETH\', \'mean\')", \'Sold\')',
 '("(\'amountInETH\', \'min\')", \'Bid\')',
 '("(\'amountInETH\', \'min\')", \'Listed\')',
 '("(\'amountInETH\', \'min\')", \'Sold\')']

In [37]:
columns_to_drop =  ['("(\'amountInETH\', \'len\')", \'Listed\')',
 '("(\'amountInETH\', \'len\')", \'Sold\')',
 '("(\'amountInETH\', \'mean\')", \'Listed\')',
 '("(\'amountInETH\', \'mean\')", \'Sold\')',
 '("(\'amountInETH\', \'min\')", \'Listed\')',
 '("(\'amountInETH\', \'min\')", \'Sold\')']
nft_bid_df.drop(columns_to_drop, axis = 1, inplace = True)

In [38]:
old_column_names = nft_bid_df.columns
new_column_names =  ['n_id','n_bid_count','n_bid_max','n_list_max','n_sold_max','n_bid_max','n_bid_min']
column_change = {old_column_names[col]:new_column_names[col] for col in range(len(old_column_names))}
nft_bid_df.rename(columns = column_change, inplace = True);

In [39]:
nft_bid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56177 entries, 0 to 56176
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   n_id         56177 non-null  object 
 1   n_bid_count  33969 non-null  float64
 2   n_bid_max    33969 non-null  float64
 3   n_list_max   55755 non-null  float64
 4   n_sold_max   34507 non-null  float64
 5   n_bid_max    33969 non-null  float64
 6   n_bid_min    33969 non-null  float64
dtypes: float64(6), object(1)
memory usage: 3.0+ MB


In [40]:
nft_bid_df.describe().T

,count,mean,std,min,25%,50%,75%,max
n_bid_count,33969.0,2.411876,2.912781,1.0,1.00000,1.00,3.0000,43.000000
n_bid_max,33969.0,1.188456,13.712631,0.1,0.22010,0.45,1.0000,2224.000000
n_list_max,55755.0,13.149142,944.759184,0.1,0.30000,0.55,1.0000,100000.000000
n_sold_max,34507.0,1.193549,13.663456,0.0,0.22035,0.45,1.0000,2224.000000
n_bid_max,33969.0,0.819407,3.645542,0.1,0.20000,0.40,0.9025,552.849126
n_bid_min,33969.0,0.623540,1.002665,0.1,0.20000,0.35,0.7770,50.000000


In [41]:
nft_bid_df.isnull().sum()

n_id               0
n_bid_count    22208
n_bid_max      22208
n_list_max       422
n_sold_max     21670
n_bid_max      22208
n_bid_min      22208
dtype: int64

## Combining all tables into flat table for data analysis

In [44]:
print("creator_df :" + str(creator_df.shape))
print("nft_sales_df :" + str(nft_sales_df.shape))
print("nft_metadata_df :" + str(nft_metadata_df.shape))
print("nft_bid_df :" + str(nft_bid_df.shape))

print("creator_nft_df :" + str(creator_nft_df.shape))
print("nft_nft_history_df :" + str(nft_nft_history_df.shape))

creator_df :(6000, 4)
nft_sales_df :(58830, 8)
nft_metadata_df :(58720, 4)
nft_bid_df :(56177, 7)
creator_nft_df :(58904, 2)
nft_nft_history_df :(327651, 2)


#### Drop records where nft_id is null in "creator_nft_df", likely due to being system transactions in processing?

In [54]:
creator_nft_df = creator_nft_df[creator_nft_df.nft_id.notnull()]
print("creator_nft_df :" + str(creator_nft_df.shape))

creator_nft_df :(58871, 2)


In [56]:
nft_combined = pd.merge(nft_sales_df, creator_nft_df, how = "left", left_on = "n_id", right_on = "nft_id", validate = "one_to_one")
nft_combined.drop('nft_id', axis = 1, inplace = True)

In [57]:
nft_combined = pd.merge(nft_combined, creator_df, how = "left", left_on = "creator_id", right_on = "c_id", validate = "many_to_one")
nft_combined.drop('creator_id', axis = 1, inplace = True)

In [58]:
nft_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58830 entries, 0 to 58829
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   n_id                  58830 non-null  object        
 1   n_isFirstSale         58830 non-null  bool          
 2   n_lastSalePriceInETH  34493 non-null  float64       
 3   n_revenue             58830 non-null  float64       
 4   n_sales               58830 non-null  float64       
 5   n_tokenIPFSPath       58830 non-null  object        
 6   n_date_minted         58830 non-null  datetime64[ns]
 7   n_rs_ratio            34486 non-null  float64       
 8   c_id                  58774 non-null  object        
 9   c_revenue             58774 non-null  float64       
 10  c_sales               58774 non-null  float64       
 11  c_rs_ratio            58774 non-null  float64       
dtypes: bool(1), datetime64[ns](1), float64(7), object(3)
memory usage: 5.4+ MB

In [97]:


nft_combined = pd.merge(nft_combined, nft_metadata_df, how = "left", left_on = "n_tokenIPFSPath", right_on = "m_tokenIPFS", validate = "many_to_one")
nft_combined.drop('m_tokenIPFS', axis = 1, inplace = True)

#### Come back to check, how can 2 nfts refer to the same IPFSObject

In [96]:
pd.options.display.max_colwidth = 200
nft_combined[nft_combined['n_tokenIPFSPath'].duplicated() == True].sort_values(by = "n_tokenIPFSPath")

,n_id,n_isFirstSale,n_lastSalePriceInETH,n_revenue,n_sales,n_tokenIPFSPath,n_date_minted,n_rs_ratio,c_id,c_revenue,c_sales,c_rs_ratio
52193,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-95057,True,NaN,0.00000,0.000,QmNdaPDyezZZjbi1yXS77sN9yhXekJvEAF7TCa6xbb2HPC/metadata.json,2021-10-07 02:06:44,NaN,0x3fddbe672f75b4af195e98ba8eae617db9e57522,0.885700,1.042000,0.850000
32266,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-950,True,NaN,0.00000,0.000,QmNgs6HUJwBNKLxjYnhkihi2yofT3PdgqM7hkipxyJbH2r/metadata.json,2021-02-24 19:37:15,NaN,0xae4b19556b24157be2dc459048a0e1ddbfd3b4c8,2.720000,3.200000,0.850000
32267,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-960,True,NaN,0.00000,0.000,QmNgs6HUJwBNKLxjYnhkihi2yofT3PdgqM7hkipxyJbH2r/metadata.json,2021-02-24 20:25:30,NaN,0xae4b19556b24157be2dc459048a0e1ddbfd3b4c8,2.720000,3.200000,0.850000
25207,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-50287,True,NaN,0.00000,0.000,QmNh9XmM3goxz4R4xLvnrhUFa6GzoU6m8RxDxXt3SrDXHa/metadata.json,2021-06-16 18:28:14,NaN,0x0adec4fe72a3b18e393b9f2a732bddecadb2b9a5,4.182850,4.921000,0.850000
17217,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-56298,True,NaN,0.00000,0.000,QmP1P9CuD4LZ9YmuZc92zMLuqm3snnFz3izRPu52ZDRWzV/metadata.json,2021-07-04 11:09:40,NaN,0x6b466c24b0651d52995a66d3e653398c308a4e96,7.075570,8.324200,0.850000
...,...,...,...,...,...,...,...,...,...,...,...,...
2209,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-79766,True,NaN,0.00000,0.000,QmfEBkrMAredGxNhMP5Mji4wJwZ2vthmmKUKPTuTUZAbZv/metadata.json,2021-09-01 09:50:21,NaN,0xeb915ec165d566ab503f95df9ace913f2cb45384,37.825000,44.500000,0.850000
13914,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-19807,False,3.000,2.55000,3.000,QmfGTUKp9M2Bri3fqiAaejcm6CLsgpT2gFUehkW84mYD2N/metadata.json,2021-04-05 22:17:20,0.85,0x103dc11cd79a6e4f0fd678c8a6821904edd19922,8.540677,10.047856,0.850000
46132,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-54477,True,NaN,0.00000,0.000,QmfVcxHUFtknVTqqgPfERDjtyZhuCQBqoo7Y5D8DMGpydY/metadata.json,2021-06-29 11:32:06,NaN,0xf80d10b752cb9bc63439a0bc7ecd111d5a0fd059,1.271430,1.495800,0.850000
10417,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-104869,False,0.385,0.32725,0.385,QmfY4mLYAaFWaYwwM3Sqxdvzih92KoTjXRgBDPV4kDga5e/metadata.json,2021-11-05 03:53:24,0.85,0xcc3f1b6bf0536bf91418075593ed72858f07f795,10.294705,13.417300,0.767271


In [98]:
nft_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58830 entries, 0 to 58829
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   n_id                  58830 non-null  object        
 1   n_isFirstSale         58830 non-null  bool          
 2   n_lastSalePriceInETH  34493 non-null  float64       
 3   n_revenue             58830 non-null  float64       
 4   n_sales               58830 non-null  float64       
 5   n_tokenIPFSPath       58830 non-null  object        
 6   n_date_minted         58830 non-null  datetime64[ns]
 7   n_rs_ratio            34486 non-null  float64       
 8   c_id                  58774 non-null  object        
 9   c_revenue             58774 non-null  float64       
 10  c_sales               58774 non-null  float64       
 11  c_rs_ratio            58774 non-null  float64       
 12  m_name                58827 non-null  object        
 13  m_desc          

In [99]:
nft_combined = pd.merge(nft_combined, nft_bid_df, how = "left", left_on = "n_id", right_on = "n_id", validate = "one_to_one")

In [100]:
nft_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58830 entries, 0 to 58829
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   n_id                  58830 non-null  object        
 1   n_isFirstSale         58830 non-null  bool          
 2   n_lastSalePriceInETH  34493 non-null  float64       
 3   n_revenue             58830 non-null  float64       
 4   n_sales               58830 non-null  float64       
 5   n_tokenIPFSPath       58830 non-null  object        
 6   n_date_minted         58830 non-null  datetime64[ns]
 7   n_rs_ratio            34486 non-null  float64       
 8   c_id                  58774 non-null  object        
 9   c_revenue             58774 non-null  float64       
 10  c_sales               58774 non-null  float64       
 11  c_rs_ratio            58774 non-null  float64       
 12  m_name                58827 non-null  object        
 13  m_desc          

In [101]:
nft_combined.shape

(58830, 21)

## Tidying up the final table

As the project is to develop a model for nft prices, only records where lastSalePriceInETH is not null and not zero is preserved.

In [102]:
nft_combined =  nft_combined[nft_combined.n_lastSalePriceInETH.notnull()]
print("After removing nulls " + str(nft_combined.shape))
nft_combined =  nft_combined[nft_combined['n_lastSalePriceInETH']!= 0]
print("After removing zeros " + str(nft_combined.shape))

After removing nulls (34493, 21)
After removing zeros (34484, 21)


Checking if data from bid table matches with nft table

In [105]:
(nft_combined['n_lastSalePriceInETH']!=nft_combined['n_sold_max']).sum() 

135

Come back to check how come the prices dont match for 135 records (check against all 3 sales columns)

In [107]:
nft_sales_mismatch = nft_combined[nft_combined['n_lastSalePriceInETH']!=nft_combined['n_sold_max']]

In [108]:
nft_combined =  nft_combined[nft_combined['n_lastSalePriceInETH'] == nft_combined['n_sold_max']]
print("After removing mismatch in sale " + str(nft_combined.shape))

After removing mismatch in sale (34349, 21)


In [112]:
nft_combined.drop(['n_sales','n_sold_max'],axis = 1, inplace = True)

In [113]:
nft_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34349 entries, 0 to 58774
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   n_id                  34349 non-null  object        
 1   n_isFirstSale         34349 non-null  bool          
 2   n_lastSalePriceInETH  34349 non-null  float64       
 3   n_revenue             34349 non-null  float64       
 4   n_tokenIPFSPath       34349 non-null  object        
 5   n_date_minted         34349 non-null  datetime64[ns]
 6   n_rs_ratio            34349 non-null  float64       
 7   c_id                  34349 non-null  object        
 8   c_revenue             34349 non-null  float64       
 9   c_sales               34349 non-null  float64       
 10  c_rs_ratio            34349 non-null  float64       
 11  m_name                34347 non-null  object        
 12  m_desc                33636 non-null  object        
 13  m_img           

In [115]:
nft_combined.describe().T

,count,mean,std,min,25%,50%,75%,max
n_lastSalePriceInETH,34349.0,1.193024,13.693330,0.0100,0.220100,0.450000,1.000000,2224.000000
n_revenue,34349.0,0.996667,11.629825,0.0085,0.187085,0.382500,0.850000,1890.400000
n_rs_ratio,34349.0,0.839462,0.076145,0.1000,0.850000,0.850000,0.850000,0.927778
c_revenue,34349.0,13.984777,73.846482,0.1000,1.672035,3.859935,9.345835,1890.400000
c_sales,34349.0,17.198369,90.362788,0.7330,2.000000,4.700200,11.355500,2224.000000
c_rs_ratio,34349.0,0.829166,0.058853,0.1000,0.850000,0.850000,0.850000,0.850000
n_bid_count,33693.0,2.408542,2.906817,1.0000,1.000000,1.000000,3.000000,43.000000
n_bid_max,33693.0,1.190137,13.767197,0.1000,0.220100,0.450000,1.000000,2224.000000
n_list_max,33937.0,9.311468,714.741790,0.1000,0.250000,0.500000,1.000000,100000.000000
n_bid_max,33693.0,0.820157,3.658895,0.1000,0.200000,0.400000,0.906275,552.849126


In [116]:
nft_combined.isnull().sum()

n_id                      0
n_isFirstSale             0
n_lastSalePriceInETH      0
n_revenue                 0
n_tokenIPFSPath           0
n_date_minted             0
n_rs_ratio                0
c_id                      0
c_revenue                 0
c_sales                   0
c_rs_ratio                0
m_name                    2
m_desc                  713
m_img                     1
n_bid_count             656
n_bid_max               656
n_list_max              412
n_bid_max               656
n_bid_min               656
dtype: int64

In [118]:
nft_combined.head(20)

,n_id,n_isFirstSale,n_lastSalePriceInETH,n_revenue,n_tokenIPFSPath,n_date_minted,n_rs_ratio,c_id,c_revenue,c_sales,c_rs_ratio,m_name,m_desc,m_img,n_bid_count,n_bid_max,n_list_max,n_bid_max,n_bid_min
0,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-24437,False,2224.0000,1890.400000,QmNeqUFf3WKcXFEHgxFCrf3t8jrxyCkev6LBtRz78Ygq9u/metadata.json,2021-04-14 22:13:06,0.850000,0xf74d1224931afa9cf12d06092c1eb1818d1e255c,1890.400000,2224.0000,0.850000,"Stay Free (Edward Snowden, 2021)","This unique, signed work combines the entirety of a landmark court decision ruling the National Security Agency's mass surveillance violated the law, with the iconic portrait of the whistleblower ...",ipfs://QmTKdyzYbT9FzAs8mB4HGiLDR7o8maRP34J5iNDFZQqndQ/nft.png,31.0,2224.0000,2.0,552.849126,2.0000
1,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100052,False,49.5002,42.075170,QmaW2ukMCqyMNg5qqoYsusgbNL9KRnFwRKCkBt6u3AiyTN/metadata.json,2021-10-19 04:56:48,0.850000,0x4666fd1f89576e2d6fb3f2ecec5eefd3e1ba6b59,1329.278945,1624.6517,0.818193,PORTAL 07,Possession of this PORTAL entitles you to membership in the TBOA club (theblocksofart.com)\n\n10000x10000,ipfs://QmYChA4jtLhiMLPhDZv1t3xGZ1uj46yG4dtzCygUjgdhz7/nft.png,8.0,49.5002,0.1,29.538775,0.1000
2,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-100995,False,33.0000,3.300000,QmdCKx9Q5b1G4vjbJ2wKSkCo3sWoUrg2hm5QTifNag5nXj/metadata.json,2021-10-21 05:35:05,0.100000,0x4666fd1f89576e2d6fb3f2ecec5eefd3e1ba6b59,1329.278945,1624.6517,0.818193,PORTAL 08,Possession of this PORTAL entitles you to membership in the TBOA club (tboa.club)\n\n444(4) Edition\n\n10000x10000,ipfs://Qmbd3CzQMBPYeYBiMWe7U2bTWCyRusEjcMB3Ck84LcDbGc/nft.png,1.0,33.0000,33.0,33.000000,33.0000
6,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-104231,False,38.5000,32.725000,QmcwsRse5TswGBcNMyzUqEERpKAdv2knwkXCab28xzXqMg/metadata.json,2021-11-02 11:12:05,0.850000,0x4666fd1f89576e2d6fb3f2ecec5eefd3e1ba6b59,1329.278945,1624.6517,0.818193,PORTAL 12,Possession of this PORTAL entitles you to membership in the TBOA club (tboa.club)\n\n10000x10000,ipfs://QmcReVCdz6CSsvhYNnrGQH93BEQhdatd7TNV1CivYqFer1/nft.png,5.0,38.5000,0.1,14.772000,0.1000
7,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-61273,False,1.5000,1.275000,QmQ2rg7edL3JMGaVa7u57crn4xyNheXGLsyUxmQX5v26eZ/metadata.json,2021-07-18 16:36:42,0.850000,0x4666fd1f89576e2d6fb3f2ecec5eefd3e1ba6b59,1329.278945,1624.6517,0.818193,STRW01,NaN,ipfs://QmQcLpW91JS184BLGe5KzB98LAynKQDdg7SPZsM7wD1Lcy/nft.png,1.0,1.5000,1.5,1.500000,1.5000
8,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-63149,False,5.0000,4.250000,QmQgj6xy2cwjnH5vpQoey4EAHXddafDb5DZiAXqpVL6fco/metadata.json,2021-07-23 04:16:25,0.850000,0x4666fd1f89576e2d6fb3f2ecec5eefd3e1ba6b59,1329.278945,1624.6517,0.818193,dєѕírєd ѕσlítudє,8000 x 8000,ipfs://QmbQn11K5DMk14FxLVwNyfJMWLZPy2biRGgnPwebzYGoxf/nft.png,9.0,5.0000,1.2,2.338900,1.2000
9,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-64683,False,1.3200,1.122000,QmRUMLZTL6pGA8FhXYZHHaZdRzpkV16tGaMigb3hRmdqza/metadata.json,2021-07-26 17:33:22,0.850000,0x4666fd1f89576e2d6fb3f2ecec5eefd3e1ba6b59,1329.278945,1624.6517,0.818193,bᎥᎾᏟᎬᏞᎬsᏆᎥᎪᏞ,8000x8000,ipfs://Qmb9oBysDQTSd8uEsBwZVqesUMVdBhQQdBvaUMPx44r5m9/nft.png,2.0,1.3200,6.0,1.260000,1.2000
10,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-65069,False,23.8800,20.298000,QmXZ24m9rrTnsvKtU5jZzquonKJq1TdQxWCYz3zfzWfSKh/metadata.json,2021-07-27 12:52:05,0.850000,0x4666fd1f89576e2d6fb3f2ecec5eefd3e1ba6b59,1329.278945,1624.6517,0.818193,PORTAL 01,Possession of this PORTAL entitles you to membership in the TBOA club (theblocksofart.com)\n\n8000x8000,ipfs://QmbDqs81sSFjDwnjRvEL9vizrJ9wXddSbw5ntLwnvJ7arL/nft.png,13.0,23.8800,5.0,12.040777,5.0000
11,0x3b3ee1931dc30c1957379fac9aba94d1c48a5405-65450,False,2.4300,2.065500,QmdXBm1C3G59kgBe43sziZ6HgziZ8DpoHHfX94fK4i137X/metadata.json,2021-07-28 13:25:56,0.850000,0x4666fd1f89576e2d6fb3f2ecec5eefd3e1ba6b59,1329.278945,1624.6517,0.818193,changes,8000x8000,ipfs://QmNyHG5Jq5eWZBdkRwu3m9n3UBAMzht3wHXy3iDHeeMQqD/nft.png,8.0,2.4300,1.2,1.750263,1.2000
12,0x3b3ee1931dc